# Golden Globes Best Actress Analysis

According to Google AI overview The Golden Globes, SAG Awards, BAFTAs, and Critics' Choice Awards are some of the awards that influence the Oscars. Therefore, I selected only these four awards for my analysis.

I will base my analysis in a range of 10 years (2014-2023)

I will probably use logistic regression


In [2]:
GOLDEN_GLOBES_url = "https://en.wikipedia.org/wiki/Golden_Globe_Award_for_Best_Actress_in_a_Motion_Picture_%E2%80%93_Drama"

In [3]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

soup = BeautifulSoup(requests.get(GOLDEN_GLOBES_url).text, 'html.parser')


In [4]:
tables = soup.find_all('table', class_='wikitable sortable')

golden_globes_2020s = tables[-1]
golden_globes_2010s = tables[-2]

In [5]:
def isWinner(row):
    return bool(row.find('td',{'style': 'background:#B0C4DE;'})) 

In [6]:
df = pd.DataFrame(columns=['ano','nome','personagem','filme','ganhou'])


for table in tables:
    rows = table.find_all('tr')
    current_year = None  # Variável para armazenar o ano atual
    current_filme = None
    
    for row in rows[1:]:
        ano_th = row.find('th', {'style': 'text-align:center;'}) 
        if not ano_th:
             ano_th = row.find('th')
        
        # Se encontrou um th com o estilo, atualiza o ano atual
        if ano_th is not None and ano_th.find('a'):
            current_year = ano_th.find('a').text 
        
        # Usa o ano atual para esta linha
        ano_atual = current_year
        #print(f"Ano para esta linha: {ano_atual}")
        
        # Processa os tds
        tds = row.find_all('td')
        nome = tds[0].find('a').text if tds[0].find('a') else None
        if nome is None:
            nome = tds[0].text.strip()
        personagem = tds[1].find('a').text if tds[1].find('a') else None
        if personagem is None:
             personagem = tds[1].text.strip()
        if len(tds) > 2:
                filme = tds[2].find('a').text if tds[2].find('a') else None
                # Atualiza o filme atual para uso em linhas futuras
                if filme:
                    current_filme = filme
                else:
                    # Usa o filme armazenado anteriormente
                    filme = current_filme
    
        ganhou = isWinner(row)
    
        df = pd.concat(
            [
                df,
                pd.DataFrame(
                    {
                        'ano': [ano_atual],
                        'nome': [nome],
                        'personagem': [personagem],
                        'filme': [filme],
                        'ganhou': [ganhou]
                    }
                )
            ]
    )

In [7]:
df

,ano,nome,personagem,filme,ganhou
0,1943,Jennifer Jones,Bernadette Soubirous,The Song of Bernadette,True
0,1944,Ingrid Bergman,Paula Alquist Anton,Gaslight,True
0,1945,Sister Mary Benedict,The Bells of St. Mary's,[3],True
0,1946,Rosalind Russell,Sister Elizabeth Kenny,Sister Kenny,True
0,1947,Rosalind Russell,Lavinia Mannon,Mourning Becomes Electra,True
...,...,...,...,...,...
0,2024,Pamela Anderson,Shelly Gardner,The Last Showgirl,False
0,2024,Angelina Jolie,Maria Callas,Maria,False
0,2024,Nicole Kidman,Romy Mathis,Babygirl,False
0,2024,Tilda Swinton,Martha Hunt,The Room Next Door,False


In [8]:
print(df[df['nome'] == 'Vanessa Kirby'])

    ano           nome    personagem              filme ganhou
0  2020  Vanessa Kirby  Martha Weiss  Pieces of a Woman  False


In [9]:
df.to_csv('data/original/golden_globes.csv', index=False)
print('done')

done
